<a href="https://colab.research.google.com/github/syh0397/Statistics_python/blob/main/14_%ED%8C%90%EB%B3%84%EB%B6%84%EC%84%9D(Discriminant_Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install and import


In [ ]:
!pip install pingouin
!pip install --upgrade pingouin
!pip install scikit_posthocs
!pip install --upgrade patsy
!pip install patsy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd 
import numpy as np 

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import sklearn 
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)

import scipy 
from scipy import  stats
import statsmodels.api as sm
import pylab as py

import pingouin as pg
import scikit_posthocs

pd.options.display.float_format = '{:.3f}'.format

## 판별분석

#### 판별분석 ? 
- 이미 분류는 되어 있는데 
- 그 분류되어 있는 집단 사이의 차이를 설명할수 있는 독립변수 X를 찾아내고 
- 이 변수들의 선형 결합으로 이루어진 식이나 함수를 통해서 집단을 분류하고 예측함  

즉, 무슨 이유로 이렇게 분류가 되었는지 판별해보는 분석방법 

In [31]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix

df = sns.load_dataset('titanic', sep=',', encoding='CP949')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.000,1,0,7.250,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.000,1,0,71.283,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.000,0,0,7.925,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.000,1,0,53.100,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.000,0,0,8.050,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000,0,0,13.000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.000,0,0,30.000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.450,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.000,0,0,30.000,C,First,man,True,C,Cherbourg,yes,True


- 인코딩하기 귀찮으니 안하려다가 가장 영향을 많이 끼치는 sex 컬럼을 빼먹을수 없었으므로 원핫인코딩을 살짝해준다. 

In [32]:
df.dropna()
df = pd.get_dummies(df)
df

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex_female,sex_male,...,deck_C,deck_D,deck_E,deck_F,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,alive_no,alive_yes
0,0,3,22.000,1,0,7.250,True,False,0,1,...,0,0,0,0,0,0,0,1,1,0
1,1,1,38.000,1,0,71.283,False,False,1,0,...,1,0,0,0,0,1,0,0,0,1
2,1,3,26.000,0,0,7.925,False,True,1,0,...,0,0,0,0,0,0,0,1,0,1
3,1,1,35.000,1,0,53.100,False,False,1,0,...,1,0,0,0,0,0,0,1,0,1
4,0,3,35.000,0,0,8.050,True,True,0,1,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000,0,0,13.000,True,True,0,1,...,0,0,0,0,0,0,0,1,1,0
887,1,1,19.000,0,0,30.000,False,True,1,0,...,0,0,0,0,0,0,0,1,0,1
888,0,3,NaN,1,2,23.450,False,False,1,0,...,0,0,0,0,0,0,0,1,1,0
889,1,1,26.000,0,0,30.000,True,True,0,1,...,1,0,0,0,0,1,0,0,0,1


In [33]:
x = df[['pclass', 'sibsp','fare','adult_male','sex_female','sex_male']]
y = df[['survived']]


lda = LDA().fit(x,y)
print(pd.DataFrame(lda.predict_proba(x)))

        0     1
0   0.970 0.030
1   0.052 0.948
2   0.242 0.758
3   0.055 0.945
4   0.951 0.049
..    ...   ...
886 0.872 0.128
887 0.036 0.964
888 0.337 0.663
889 0.696 0.304
890 0.951 0.049

[891 rows x 2 columns]


#### LDA ?
- 투영(project) 후 두 클래스 간 분산은 최대한 크게 가져가고, 클래스 내부의 분산은 최대한 작게 가져가는 방식
-  LDA는 특정 공간상에서 클래스 분리를 최대화하는 축을 찾기 위해 클래스 간 분산(between-class scatter)과 클래스 내부 분산(within-class scatter)의 비율을 최대화하는 방식으로 차원을 축소

- 분산이 크다 = 정보량이 많다 = 정보량을 잃지 않게 분류 잘해야 한다. 


1. 예측점수로 판별식에 의해서 도출됨
2. 계체별 예측 점수를 도출해줌 

In [34]:
cf_m = pd.DataFrame(confusion_matrix(y, lda.predict(x)))
cf_m.columns = ['pre 0', 'pre 1']
cf_m.index = ['re 0', 're 1']

print(cf_m)

      pre 0  pre 1
re 0    481     68
re 1     90    252


- confusion_matrix (혼동행렬) 보면서 잘 분류 되었는지 본다. 
- pre = 예측값
- re = real 실제값 
- 0인데 0 이다 , 1인데 1이다 => 제대로 분류된 부분 
  - 481 + 252 는 정확
- 68, 90 은 오류 